In [5]:
import os
import rasterio
import geopandas as gpd
import pandas as pd
from rasterio.features import shapes
from shapely.geometry import shape
from shapely.geometry import Point
from shapely.geometry import shape
from shapely.ops import unary_union
from shapely.geometry import Point
from shapely.geometry import Polygon
from shapely.geometry import MultiPolygon
from rtree import index

In [6]:
def fill_holes(row):
    if row['geometry'].is_empty or not row['geometry'].is_valid:
        return row['geometry']
    if row['geometry'].geom_type == 'Polygon':
        return row['geometry']
    holes = list(row['geometry'].interiors)
    
    exterior = row['geometry'].exterior
    
    filled_geometry = Polygon(exterior, holes)
    
    return filled_geometry

In [7]:
def raster_to_vector(input_folder, output_file):
    # Create an empty GeoDataFrame to store all features
    all_features = []

    # Get all TIFF image files in the input folder
    tif_files = [f for f in os.listdir(input_folder) if f.endswith('.tif')]

    for tif_file in tif_files:
        input_path = os.path.join(input_folder, tif_file)

        # Open the TIFF image file
        with rasterio.open(input_path) as src:
            # Read TIFF data
            image = src.read(1)

            # Create a shape list to store vector features
            shapes_list = list(shapes(image, mask=None, transform=src.transform))

            # Filter out parts with a value of 1
            shapes_list = [shape(geometry) for geometry, value in shapes_list if value == 1]

            # Create a GeoDataFrame to store features of the current TIFF file
            gdf = gpd.GeoDataFrame({'geometry': shapes_list}, crs=src.crs)  # Set the coordinate reference system

            # Calculate the area of each feature and add it to the GeoDataFrame
            gdf['area_km2'] = gdf['geometry'].area / 1e6  # Convert to square kilometers

            # Filter features with an area greater than or equal to 0.1 square kilometers
            gdf = gdf[gdf['area_km2'] >= 0.1]

            # Add the features of the current TIFF file to all features
            all_features.append(gdf)

    merged_gdf = gpd.GeoDataFrame(pd.concat(all_features, ignore_index=True), crs=src.crs)

    merged_gdf['geometry'] = merged_gdf['geometry'].apply(lambda geom: geom.buffer(45, cap_style=3))

    # Spatial intersection check and merge
    merged_gdf['group'] = None
    group_count = 0

    for index, row in merged_gdf.iterrows():
        if row['group'] is None:
            # Find features that intersect with the current feature
            intersection = merged_gdf[merged_gdf['geometry'].intersects(row['geometry'])]

            if len(intersection) > 1:
                # Update group number
                group_count += 1
                merged_gdf.loc[intersection.index, 'group'] = group_count

    merged_gdf['group'] = merged_gdf['group'].fillna(0)  # Set group number to 0 for ungrouped features
    merged_gdf = merged_gdf.dissolve(by='group')
    merged_gdf['geometry'] = merged_gdf['geometry'].apply(lambda geom: geom.buffer(-40, cap_style=3))

    single_polygons = []
    for geom in merged_gdf['geometry']:
        if geom.geom_type == 'MultiPolygon':
            # Break down compound polygons into single polygons
            single_polygons.extend(list(geom))
        else:
            single_polygons.append(geom)
    merged_gdf = gpd.GeoDataFrame({'geometry': single_polygons}, crs=merged_gdf.crs)

    for index, row in merged_gdf.iterrows():
        if row['geometry'].interiors:
            print('fill hole when raster!!!!!')
            exterior = row['geometry'].exterior
            interiors = list(row['geometry'].interiors)
            filled_geom = Polygon(exterior, interiors)
            merged_gdf.at[index, 'geometry'] = filled_geom


    simplified_gdf = merged_gdf.copy()

    simplified_gdf['geometry'] = simplified_gdf['geometry'].simplify(10.0)  # simplify polygon

    # Calculate the length of each polygon
    simplified_gdf['length'] = simplified_gdf['geometry'].length

    # Calculate the bounding box for each polygon
    bounds = simplified_gdf['geometry'].bounds

    # Calculate the width of each polygon (the difference along the x-axis)
    simplified_gdf['width'] = bounds['maxx'] - bounds['minx']

    # Generate two radius fields, using half of the length and width
    simplified_gdf['radius_length'] = simplified_gdf['length'] / 2
    simplified_gdf['radius_width'] = simplified_gdf['width'] / 2

    # Calculate the ratio of the two radius fields and store it in a new column
    simplified_gdf['radius_ratio'] = simplified_gdf['radius_length'] / simplified_gdf['radius_width']

    # Select the part where radius_ratio is less than 10
    filtered_gdf = simplified_gdf[simplified_gdf['radius_ratio'] < 11]
    # save to GPKG
    

    for index, row in filtered_gdf.iterrows():
        if row['geometry'].is_valid and row['geometry'].interiors:
            
            
            print('find hole when vector !!!!!')
            # Get the exterior ring of the polygon
            exterior = row['geometry'].exterior

            # Get all interior rings (holes) of the polygon
            interiors = list(row['geometry'].interiors)

            # Create a new polygon by merging the exterior and interior rings
            filled_polygon = Polygon(exterior)
            
            # Take the intersection of the filled polygon with the original polygon
            # to remove any overlapping areas
            #filled_polygon = filled_polygon.intersection(row['geometry'])

            # Add the filled polygon to the new GeoDataFrame
            filtered_gdf = filtered_gdf.append({'geometry': filled_polygon}, ignore_index=True)
        else:
            # If the polygon is invalid or has no interior rings, add the original polygon
            filtered_gdf = filtered_gdf.append({'geometry': row['geometry']}, ignore_index=True)
    filtered_gdf = filtered_gdf.unary_union

    filtered_gdf = gpd.GeoDataFrame(geometry=[filtered_gdf], crs=merged_gdf.crs)
    filtered_gdf_exploded = filtered_gdf.explode()
    

    #filtered_gdf_exploded['geometry'] = filtered_gdf_exploded['geometry'].translate(xoff=150, yoff=-150) # We had an overall offset of about 150 meters after splitting the image in the preprocessing steps. So translate are used to fix this problem. 
    filtered_gdf_exploded.to_file(output_file, driver="GPKG")

In [8]:
#input_folder =r'E:\ACM\Final_ChongCi\FinalOutPut\FinalResult_0731' #E:\ACM\Final_ChongCi\FinalOutPut\FinalResult_0603
input_folder =r'E:\ACM\Final_ChongCi\FinalOutPut\FinalResult_0825'



# input det img file path  Run Four time ！！！！！！！
output_vector = r'E:\ACM\Final_ChongCi\FinalOutPut\PredictLake_0825.gpkg' # output gpkg file

raster_to_vector(input_folder, output_vector)

D:\Software\anaconda\envs\acmtensorflow\lib\site-packages\ipykernel_launcher.py:60: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
D:\Software\anaconda\envs\acmtensorflow\lib\site-packages\ipykernel_launcher.py:60: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.


fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
fill hole when raster!!!!!
find hole when vector !!!!!
find hole when vector !!!!!
find hole when vector !!!!

D:\Software\anaconda\envs\acmtensorflow\lib\site-packages\ipykernel_launcher.py:125: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
